## JG Thoughts 
- Unemployment rate vs employment #s? Look at change in UE rate or total as of certain point in time
- Inc_Per_CAp_2022 -> typo?
- Population Density per mi² -> what yr? Also is string
- NA values in NJ
- Convert weeks into %?

In [85]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
import statsmodels.api as sm

In [54]:
df = pd.read_csv('Data/Merged_state_data.csv')

In [55]:
df.head()

,Location,Employment_2020,Employment_2021,Employment_2022,Inc_Per_Cap_2020,Inc_Per_Cap_2021,Inc_Per_CAp_2022,Life_Exp_2020,Life_Exp_2019,Life_Exp_2018,...,Exc_deaths_2019,Exc_deaths_2020,Exc_deaths_2021,Exc_deaths_2022,Exc_count_2017,Exc_count_2018,Exc_count_2019,Exc_count_2020,Exc_count_2021,Exc_count_2022
0,Alabama,2671005,2769464,2869931,45887,50059,50916,73.2,75.2,75.1,...,945,20726,29283,14805,3,15,0,99,106,87
1,Alaska,430840,443047,457687,61898,65662,68635,76.6,77.7,78.0,...,639,1422,3483,1783,6,6,3,34,75,42
2,Arizona,3920033,4086802,4287595,52133,56420,58442,76.3,78.8,78.7,...,1785,30955,40347,20653,9,15,0,89,114,77
3,Arkansas,1639829,1686444,1755536,47147,51636,52618,73.8,75.7,75.6,...,1434,11285,15391,8969,12,12,0,66,93,64
4,California,23154091,23934549,25300974,70061,76991,77036,79.0,80.9,80.8,...,1029,93875,133206,89195,18,12,0,94,104,106


In [56]:
# df.columns
df.dtypes

Location                       object
Employment_2020                 int64
Employment_2021                 int64
Employment_2022                 int64
Inc_Per_Cap_2020                int64
Inc_Per_Cap_2021                int64
Inc_Per_CAp_2022                int64
Life_Exp_2020                 float64
Life_Exp_2019                 float64
Life_Exp_2018                 float64
Employer_2019                 float64
Non-Group_2019                float64
Medicaid_2019                 float64
Medicare_2019                 float64
Military_2019                 float64
Uninsured_2019                float64
Employer_2021                 float64
Non-Group_2021                float64
Medicaid_2021                 float64
Medicare_2021                 float64
Military_2021                 float64
Uninsured_2021                float64
Population Density per mi²     object
2010 Population                 int64
2020 Population                 int64
flu_vaccination_rate_2019     float64
asthma_preva

In [67]:
df['excess_deaths_total_weeks'] =  df['Exc_count_2020'] + df['Exc_count_2021'] + df['Exc_count_2022']


df = df[['Inc_Per_CAp_2022', 'Life_Exp_2020', 'Uninsured_2021', 'flu_vaccination_rate_2019', 'asthma_prevalence','excess_deaths_total_weeks',
        'cardiac_mortality_rate', 'high_bp_prevalence', 'copd_prevalence', 'kidney_disease_prevalence', 'diabetes_prevalence', 'Active DO Rate']]

In [68]:
# Drop NAs in df
df.dropna(inplace=True)

In [69]:
# y variable will be # of weeks with excess deaths in 2020, 2021 and 2022
y = df['excess_deaths_total_weeks']

In [70]:
# X variables
X = df[['Inc_Per_CAp_2022', 'Life_Exp_2020', 'Uninsured_2021', 'flu_vaccination_rate_2019', 'asthma_prevalence',
        'cardiac_mortality_rate', 'high_bp_prevalence', 'copd_prevalence', 'kidney_disease_prevalence', 'diabetes_prevalence', 'Active DO Rate']]

In [83]:
# X.head()

## Linear Regression (LR)

In [77]:
X = sm.add_constant(X)
ols = sm.OLS(y, X).fit()

In [78]:
ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                OLS Regression Results                               
=====================================================================================
Dep. Variable:     excess_deaths_total_weeks   R-squared:                       0.467
Model:                                   OLS   Adj. R-squared:                  0.309
Method:                        Least Squares   F-statistic:                     2.948
Date:                       Tue, 31 Oct 2023   Prob (F-statistic):            0.00672
Time:                               16:47:04   Log-Likelihood:                -262.11
No. Observations:                         49   AIC:                             548.2
Df Residuals:                             37   BIC:                             570.9
Df Model:                                 11                                         
Covariance Type:                   nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                       130.3773   1009.599      0.129      0.898   -1915.265    2176.020
Inc_Per_CAp_2022             -0.0001      0.002     -0.062      0.951      -0.003       0.003
Life_Exp_2020                 1.8377     11.606      0.158      0.875     -21.678      25.354
Uninsured_2021              691.8334    414.346      1.670      0.103    -147.712    1531.379
flu_vaccination_rate_2019    -3.6849      2.518     -1.464      0.152      -8.786       1.416
asthma_prevalence            -2.2474     10.538     -0.213      0.832     -23.599      19.104
cardiac_mortality_rate        0.0796      0.604      0.132      0.896      -1.144       1.303
high_bp_prevalence           -3.8579      5.833     -0.661      0.512     -15.676       7.960
copd_prevalence               4.1030     13.977      0.294      0.771     -24.218      32.424
kidney_disease_prevalence     9.6891     26.001      0.373      0.712     -42.993      62.371
diabetes_prevalence          13.4773     12.278      1.098      0.279     -11.400      38.354
Active DO Rate               -0.8568      0.863     -0.993      0.327      -2.605       0.892
==============================================================================
Omnibus:                        0.591   Durbin-Watson:                   1.877
Prob(Omnibus):                  0.744   Jarque-Bera (JB):                0.172
Skew:                           0.124   Prob(JB):                        0.917
Kurtosis:                       3.151   Cond. No.                     7.77e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.77e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [88]:
y = df['excess_deaths_total_weeks']
X = df[['Inc_Per_CAp_2022', 'Life_Exp_2020', 'Uninsured_2021', 'flu_vaccination_rate_2019', 'asthma_prevalence',
        'cardiac_mortality_rate', 'high_bp_prevalence', 'copd_prevalence', 'kidney_disease_prevalence', 'diabetes_prevalence', 'Active DO Rate']]

In [89]:
# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [90]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [97]:
print(f'Train Score: {lr.score(X_train, y_train)}')
print(f'Train Score: {lr.score(X_test, y_test)}')

Train Score: 0.619299773081303
Train Score: -1.2180693891412502


In [99]:
df['excess_deaths_total_weeks']

0     292
1     151
2     280
3     223
4     304
5     280
6     136
7     126
8     274
9     351
10     81
11    207
12    285
13    220
14    152
15    156
16    235
17    258
18    136
19    148
20    113
21    232
22    144
23    286
24    194
25    171
26    136
27    190
28    101
30    246
31    177
32    275
33     86
34    183
35    231
36    258
37    176
38     85
39    284
40     86
41    250
42    332
43    193
44    156
45    284
46    242
47    213
48    175
49    143
Name: excess_deaths_total_weeks, dtype: int64